# Highway 145 Verified Landslides

## Run Google earth Engine (GEE)

In [1]:
import ee
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AbUR2VNuukOSK0gZIGZ_ERRp2XycwTgsgr-W7U4JaefBIQ-Os5W-S534tkg

Successfully saved authorization token.


## Data source

1. [Copernicus_S1_GRD](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S1_GRD) by European Space Agency(ESA)
2. USGS NAIP Imagery NDVI (GEE basemap layer)
3. USGS NAIP Imagery False Color
4. Verified landslides of North America collected by Earth Lab at the University of Colorado Boulder

## Import libraries and packages

In [ ]:
# Required libraries and packages
import os
import json
import earthpy as et
import pandas as pd
import datetime
import pathlib
import folium
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import ee
import geemap

import shapely.geometry as sgeo
import IPython.display as disp
import geemap.foliumap as geemap

from shapely.geometry import Point
from src.det import det
from src.create_geodataframe import create_geodataframe
from scipy.stats import chi2

## Set working directory

In [3]:
# Change directory to landslide-detect data path
data_path = os.path.join(et.io.HOME, "earth-analytics", "landslide-detect")
if os.path.exists(data_path):
    os.chdir(data_path)
else:
    os.makedirs(data_path)
    print('The new directory is created!')
    os.chdir(data_path)

data_path

'C:\\Users\\nasim\\earth-analytics\\landslide-detect'

In [4]:
%%bash
find .

.
./landslides.verified.csv


## Create dataframe

In [5]:
# Create DataFrame and open landslide file

landslide_gdf = gpd.read_file('landslides.verified.csv')
landslide_gdf.head(2)

,slide.id,slide.date,location,type,trigger,size,lon,lat,location_accuracy,event_title,admin_division_name,ge.lat,ge.lon,is.exact,slide.index,geometry
0,8321,2015-09-21T00:00:00Z,"Birken, BC, Canada",landslide,downpour,medium,-122.6205,50.479,5km,"Birken, BC, Canada",British Columbia,NA,NA,FALSE,1,None
1,7757,2015-12-07T18:00:00Z,Edmonds,mudslide,rain,medium,-122.3805278,47.70679444,5km,Edmonds,Washington,47.70679444,-122.3805278,TRUE,2,None


In [6]:
# Extract verified landslides of Highway 145, Colorado
Highway145_df = landslide_gdf[landslide_gdf['location'].str.contains
                              ('Highway 145')]
Highway145_df

,slide.id,slide.date,location,type,trigger,size,lon,lat,location_accuracy,event_title,admin_division_name,ge.lat,ge.lon,is.exact,slide.index,geometry
48,9114,2016-07-20T14:30:00Z,"Highway 145 between Sawpit and Placerville, Sa...",mudslide,downpour,medium,-108.0216,37.9939,5km,"Highway 145 between Sawpit and Placerville, Sa...",Colorado,NA,NA,FALSE,62,None
57,9112,2016-07-20T14:30:00Z,"Highway 145 between Sawpit and Placerville, Sa...",mudslide,downpour,medium,-108.0336,37.9979,5km,"Highway 145 between Sawpit and Placerville, Sa...",Colorado,NA,NA,FALSE,72,None
60,9113,2016-07-20T14:30:00Z,"Highway 145 between Sawpit and Placerville, Sa...",mudslide,downpour,medium,-108.0294,37.9948,5km,"Highway 145 between Sawpit and Placerville, Sa...",Colorado,NA,NA,FALSE,75,None
63,9116,2016-07-20T14:30:00Z,"Highway 145 between Sawpit and Placerville, Sa...",mudslide,downpour,medium,-108.012,37.9933,5km,"Highway 145 between Sawpit and Placerville, Sa...",Colorado,NA,NA,FALSE,78,None
67,9110,2016-07-20T14:30:00Z,"Highway 145 between Sawpit and Placerville, Sa...",mudslide,downpour,medium,-108.044,38.0072,5km,"Highway 145 between Sawpit and Placerville, Sa...",Colorado,NA,NA,FALSE,82,None
70,9117,2016-07-20T14:30:00Z,"Highway 145 between Sawpit and Placerville, Sa...",mudslide,downpour,medium,-108.0364,38.0008,5km,"Highway 145 between Sawpit and Placerville, Sa...",Colorado,NA,NA,FALSE,85,None
71,9109,2016-07-20T14:30:00Z,"Highway 145 between Sawpit and Placerville, Sa...",mudslide,downpour,medium,-108.0462,38.009,5km,"Highway 145 between Sawpit and Placerville, Sa...",Colorado,NA,NA,FALSE,86,None
84,9107,2016-07-20T14:30:00Z,"Highway 145 between Sawpit and Placerville, Sa...",mudslide,downpour,medium,-108.0511,38.0131,5km,"Highway 145 between Sawpit and Placerville, Sa...",Colorado,NA,NA,FALSE,100,None
96,9108,2016-07-20T14:30:00Z,"Highway 145 between Sawpit and Placerville, Sa...",mudslide,downpour,medium,-108.0488,38.0115,5km,"Highway 145 between Sawpit and Placerville, Sa...",Colorado,NA,NA,FALSE,114,None
105,9115,2016-07-20T14:30:00Z,"Highway 145 between Sawpit and Placerville, Sa...",mudslide,downpour,medium,-108.0164,37.9926,5km,"Highway 145 between Sawpit and Placerville, Sa...",Colorado,NA,NA,FALSE,123,None


In [7]:
# Extract verified large landslides
large_ls = landslide_gdf[landslide_gdf['size'].str.contains
                              ('large')]
large_ls.describe()

,slide.id,slide.date,location,type,trigger,size,lon,lat,location_accuracy,event_title,admin_division_name,ge.lat,ge.lon,is.exact,slide.index,geometry
count,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,0
unique,20,18,20,4,4,2,20,20,3,20,10,5,5,2,20,0
top,8728,2015-10-16T00:00:00Z,Broadmoor Bluffs neighborhood,landslide,rain,large,-104.8318,38.7612,1km,Broadmoor Bluffs neighborhood,NA,NA,NA,FALSE,15,NaN
freq,1,3,1,9,8,18,1,1,7,1,8,16,16,16,1,NaN


In [8]:
# Display all verified Highway 145 landslides
large_ls_map = folium.Map(
    location=[38.0021, -108.0372],
    zoom_start=4,
    width=1000,
    height=600,
    tiles='Stamen terrain')


for index, row in large_ls.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=row['slide.id'],
        icon=folium.Icon(color="red")
    ).add_to(large_ls_map)

large_ls_map

## Create interactive map of landslides of AOI (Highway 145)

In [9]:
# Display all verified Highway 145 landslides
landslide_map = folium.Map(
    location=[38.0021, -108.0372],
    zoom_start=14,
    width=1000,
    height=600,
    tiles='Stamen terrain')


for index, row in Highway145_df.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=row['slide.id'],
        icon=folium.Icon(color="red")
    ).add_to(landslide_map)

landslide_map

## AOI (Area of Interest)

In [10]:
# Make split view map half Satellite and half NAIP Imagery False Color

split_roi = geemap.Map(location=[38.0021, -108.0372], zoom_start=14)
split_roi.add_basemap('SATELLITE')
split_roi.add_basemap('USGS NAIP Imagery False Color')
# Map.add_basemap('USGS NAIP Imagery NDVI')

split_roi.split_map(left_layer='SATELLITE',
                    right_layer='USGS NAIP Imagery False Color')
split_roi

## Collect SAR Images

In [11]:
# Sentinel-1 images availability of Highway 145 landslide area
event_date = ee.Date('2016-07-20')

# Select one year surrounding the event
start_date = event_date.advance(-180, 'days')
end_date = event_date.advance(180, 'days')

'Sentinel-1 image data range is between {} and {}'.format(
    start_date.format('YYYY-MM-dd').getInfo(),
    end_date.format('YYYY-MM-dd').getInfo())

'Sentinel-1 image data range is between 2016-01-22 and 2017-01-16'

In [12]:
# define the center coordinates of region of interest
center_point = ee.Geometry.Point([-108.0372, 38.0021])

# Create an Earth Engine Bbox using the center coordinates and dimensions
width_degrees = 1000
height_degrees = 600

bbox = center_point.buffer(width / 2).bounds()

# Print the bounding box coordinates with four decimal places
print("Bounding box coordinates: ", [[round(x, 4) for x in coords] for coords
                                     in bbox.coordinates().getInfo()[0]])

bbox

NameError: name 'width' is not defined

In [13]:
# Collect and filter Sentinel-1 images by time and region of interest
start_date = '2016-01-22'
end_date = '2017-01-16'
sentinel_1 = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
              .filterBounds(bbox)
              .filterDate(start_date, end_date)
              .filter(ee.Filter.listContains('transmitterReceiverPolarisation'
                                             ,'VV'))
              .filter(ee.Filter.listContains('transmitterReceiverPolarisation'
                                             ,'VH'))
             )

image_collection = sentinel_1.filter(ee.Filter.eq('orbitProperties_pass', 
                                                  'ASCENDING'))

orbit_num = (sentinel_1.aggregate_array('relativeOrbitNumber_start')
             .getInfo())
if orbit_num:
    orbit_num = orbit_num[0]

look_angle = (sentinel_1.aggregate_array('orbitProperties_pass')
              .getInfo())
if look_angle and len(look_angle) > 0:
    look_angle = look_angle[0]

print('The Relative Orbit Number for ROI is: ', orbit_num)
print('The orbitology is: ', look_angle)

image_collection

NameError: name 'bbox' is not defined

In [14]:
# Convert collection to a list

def clip_img(img):
    """Clips a list of images."""
    return ee.Image(img).clip(bbox)

im_list = image_collection.toList(image_collection.size())
im_list = ee.List(im_list.map(clip_img))

im_list.length().getInfo()

NameError: name 'image_collection' is not defined

In [15]:
# Print full timestamp list
timestamplist = (image_collection.aggregate_array('2016-07-20')
                        .map(lambda d: ee.String('T').cat(ee.String(d)))
                        .getInfo())

timestamplist


NameError: name 'image_collection' is not defined

In [ ]:
# Define the visualization parameters
vis_params = {
    'min': 0,
    'max': 3000
}

# Add the layer to the map
Map = geemap.Map()
Map.addLayer(image_collection, vis_params, 'custom visualization')
Map.add_basemap('SATELLITE')
Map

## Change maps